In [ ]:
import pandas
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif
from collections import Counter
import time
import seaborn as sns
from matplotlib import pyplot as plt

import tensorflow as tf
#from keras.utils import to_categorical
import feather
import random
from datetime import *
import warnings
warnings.filterwarnings("ignore")

In [ ]:
FILENAME = "Datasets/final_datasets/merged_data"

In [ ]:
data = feather.read_dataframe(f"{FILENAME}.feather")

In [ ]:
# Remove those one-hot encoded columns in the dataset created by Abhi

def in_name(c, one_hots):
    for col_name in one_hots:
        if c.startswith(f"{col_name}_"):
            return True
    return False

one_hots = ['Wind_Direction', 'Timezone','State', 'Weather_Condition']
remove = [col for col in data.columns if in_name(col, one_hots)]
df = data.drop(remove, axis=1)


In [ ]:
# Remove missing values
df = df.dropna().reset_index(drop=True)
print(len(df))

In [ ]:
df['Severity'] = df['Severity'].astype(int)

# Clean categorical features

### Clean wind direction

* "CALM" = ["Calm", "CALM"]
* "W" = ["W", "West", "WSW", "WNW"] 
* "S" = ["S", "South", "SSW", "SSE"] 
* "N" = ["N", "North", "NNW", "NNE"] 
* "E" = ["E", "East", "ESE", "ENE"]
* "VAR" = ["VAR", "Variable"]

In [ ]:
df.loc[df['Wind_Direction']=='Calm','Wind_Direction'] = 'CALM'
df.loc[(df['Wind_Direction']=='West')|(df['Wind_Direction']=='WSW')|(df['Wind_Direction']=='WNW'),'Wind_Direction'] = 'W'
df.loc[(df['Wind_Direction']=='South')|(df['Wind_Direction']=='SSW')|(df['Wind_Direction']=='SSE'),'Wind_Direction'] = 'S'
df.loc[(df['Wind_Direction']=='North')|(df['Wind_Direction']=='NNW')|(df['Wind_Direction']=='NNE'),'Wind_Direction'] = 'N'
df.loc[(df['Wind_Direction']=='East')|(df['Wind_Direction']=='ESE')|(df['Wind_Direction']=='ENE'),'Wind_Direction'] = 'E'
df.loc[df['Wind_Direction']=='Variable','Wind_Direction'] = 'VAR'
print("Wind Direction after simplification: ", df['Wind_Direction'].unique())

### Clean weather condition

In [ ]:
Counter(df['Weather_Condition'])

https://www.timeanddate.com/weather/glossary.html <br>

**Create features according the different weather conditions** <br>
Scattered clouds, partly cloudy = PARTLY CLOUDY <br>
Overcast, Mostly cloudly, Cloudy = CLOUDY <br>
Fair, Clear = CLEAR <br>
Snow (light snow, heavy snow), and wintry mix  = SNOW <br>
drizzle, light rain/drizzle, light freezing rain/drizzle = LIGHT RAIN <br>
Heavy Rain, heavy thunderstorms, heavy t-storm = HEAVY RAIN <br>
Haze, fog, mist, smoke = fog     *(see https://www.worldatlas.com/articles/what-are-the-differences-between-mist-haze-and-fog.html)*<br>
Rain, light thunderstorms, thunderstorms, t-storm, thunder = RAIN <br>

In [ ]:
def get_condition(string):
    
    conditions = ['Clear', 'Partly Cloudy', 'Cloudy', 'Snow', 'Light Rain', 'Heavy Rain', 'Rain', 'Fog'] 
    strings = [['clear', 'fair'],
              ['partly cloudy', 'scattered clouds'],
              ['overcast', 'mostly cloudy', 'cloudy'],
              ['snow', 'wintry mix'],
              ['drizzle', 'light rain', 'light freezing rain'],
              ['heavy rain', 'heavy thunderstorms', 'heavy t-storm'],
              ['rain', 'thunderstorms', 't-storm', 'thunder', 'showers'],
              ['fog', 'haze', 'mist', 'smoke']]

    for j, k in enumerate(strings):
        for val in k:
            if val in string.lower():
                return conditions[j]
    return 'Others'

In [ ]:
res = []
for i in df['Weather_Condition'].values:
    res.append(get_condition(i))

df['Condition'] = res
df[['Condition', 'Weather_Condition']]

In [ ]:
conditions = ['Clear', 'Partly Cloudy', 'Cloudy', 'Snow', 'Light Rain', 'Heavy Rain', 'Rain', 'Fog', 'Others'] 

for i in conditions:
    print(i, ":")
    print(set(df[df['Condition']==i]['Weather_Condition']))
    print(sum(df['Condition']==i))
    print("\n")
    
df = df[df['Condition']!='Others']

In [ ]:
# One hot encode ['Wind_Direction', 'Timezone', 'State', 'Condition']

one_hots = ['Wind_Direction', 'Timezone','State', 'Condition']
oh = pd.DataFrame()
for c in one_hots:
    dummies = pd.get_dummies(df[c], prefix=c)
    oh = pd.concat([oh, dummies], axis=1)

df = pd.concat([df, oh], axis=1)

In [ ]:
# Save this dataframe
df = df.reset_index(drop = True)
df.to_feather("{}_v1.feather".format(FILENAME))

In [ ]:
df = feather.read_dataframe("{}_v1.feather".format(FILENAME))

#### With numeric attributes

In [ ]:
# Filter the data
#BUM NOEXIT ROUNABOUT GIVEWAY removed due to badly distributed data
drop = ['Unnamed: 0', 'ID', 'Start_Lat', 'Start_Lng', 'Start_Time', 'End_Time', 'Weather_Timestamp', 'Description', 
        'Distance(mi)', 
        'County', 'City', 'Airport_Code', 
        'Wind_Direction', 'Timezone', 'State', 'Condition', 'Weather_Condition',
        'Street', 'Bump', 'No_Exit', 'Roundabout', 'Give_Way', 'Traffic_Calming',
        "Wind_Direction"]

df = df.drop(drop, axis=1).reset_index(drop=True)

# Convert to feather file format
df.to_feather("{}_numeric_v2.feather".format(FILENAME))

In [ ]:
del(df)

#### without numeric attributes

In [ ]:
# Filter the data
#BUM NOEXIT ROUNABOUT GIVEWAY removed due to badly distributed data
drop = ['Unnamed: 0', 'ID', 'Start_Lat', 'Start_Lng', 'Start_Time', 'End_Time', 'Weather_Timestamp', 'Description', 
        'Distance(mi)', 
        'County', 'City', 'Airport_Code', 
        'Wind_Direction', 'Timezone', 'State', 'Condition', 'Weather_Condition',
        'Street', 'Bump', 'No_Exit', 'Roundabout', 'Give_Way', 'Traffic_Calming',
       'Temperature(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Direction', 'Wind_Speed(mph)', 'Precipitation(in)',
       'Population_County', 'Drive_County', 'Transit_County', 'Walk_County', 'MedianHouseholdIncome_County']

df = df.drop(drop, axis=1).reset_index(drop=True)

# Convert to feather file format
df.to_feather("{}_v2.feather".format(FILENAME))

# clears memory allocated to df
df = 0 

In [ ]:
# Clear memory
del(df)

# Model

### Finding best neuron size

In [ ]:
data = feather.read_dataframe("{}_v2.feather".format(FILENAME))
EPOCHS = 5
BATCH_SIZE = 128

# One hot encode labels
n = len(data)
labels = data['Severity']
y = np.zeros((n, 4))
for i in range(n):
    y[i, labels[i]-1] = 1
    
X = data.drop('Severity', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=30034)

max_neurons = X_train.shape[1]
neuron_size = range(max_neurons,4,-10)

for i in neuron_size:
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(i, activation=tf.nn.swish,input_shape=(X_train.shape[1],)))
    model.add(tf.keras.layers.Dense(4, activation=tf.nn.sigmoid))

    # Compile and print out summary of model
    model.compile(loss='categorical_crossentropy', 
              optimizer='adam',
             metrics=['accuracy'])
    
    
    print("==================================================")
    print("Number of Neurons: ",i)
    print("==================================================")
    model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE)
    model.evaluate(X_test, y_test)

### Model evaluation with numeric

In [ ]:
from sklearn.model_selection import StratifiedKFold

data = feather.read_dataframe("{}_numeric_v2.feather".format(FILENAME))
labels = data['Severity']
X = data.drop('Severity', axis=1)

scaler=MinMaxScaler(feature_range=(0,1))
X = scaler.fit_transform(X)

skf = StratifiedKFold(n_splits=3, random_state=30034, shuffle=False)

fold_no = 1
accs = []  # store each accuracy


for train_index, test_index in skf.split(X, labels):

    # One hot encode labels
    print("FOLD NUMBER = ", str(fold_no))


    X_train = X[train_index]
    X_test = X[test_index]
    
    
    y_train = np.zeros((len(X_train), 4))
    for i in range(len(X_train)):
        y_train[i, labels[i]-1] = 1

    y_test = np.zeros((len(X_test), 4))
    for i in range(len(X_test)):
        y_test[i, labels[i]-1] = 1

    EPOCHS = 5
    BATCH_SIZE = 128

    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(70, activation=tf.nn.swish,input_shape=(X_train.shape[1],)))
    model.add(tf.keras.layers.Dense(4, activation=tf.nn.sigmoid))

    # Compile and print out summary of model
    model.compile(loss='categorical_crossentropy', 
                  optimizer='adam',
                 metrics=['accuracy'])

    model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE)
    (test_loss, test_acc) = model.evaluate(X_test, y_test)
    print('Accuracy:', test_acc)
    accs.append(test_acc)
    print("============================================")
    fold_no+=1

### Model evaluation without numeric

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

p = []
t = []
for i in y_preds:
    p.append(np.argmax(i)+1)

for i in y_test:
    t.append(np.argmax(i)+1)    

print("accuracy:", accuracy_score(t, p))

cm=confusion_matrix(t,p)
cm = pd.DataFrame(cm, index = [i for i in "1234"],
                  columns = [i for i in "1234"])
cm.index.name = 'Actual'
cm.columns.name = 'Predicted'
sns.heatmap(cm, annot=True, 
            cmap='Blues', cbar=False)
plt.show()

In [ ]:
from sklearn.model_selection import StratifiedKFold

data = feather.read_dataframe("{}_v2.feather".format(FILENAME))
labels = data['Severity']
X = data.drop('Severity', axis=1)

scaler=MinMaxScaler(feature_range=(0,1))
X = scaler.fit_transform(X)

skf = StratifiedKFold(n_splits=3, random_state=30034, shuffle=False)

fold_no = 1
accs = []  # store each accuracy


for train_index, test_index in skf.split(X, labels):

    # One hot encode labels
    print("FOLD NUMBER = ", str(fold_no))


    X_train = X[train_index]
    X_test = X[test_index]

    y_train = np.zeros((len(X_train), 4))
    for i in range(len(X_train)):
        y_train[i, labels[i]-1] = 1

    y_test = np.zeros((len(X_test), 4))
    for i in range(len(X_test)):
        y_test[i, labels[i]-1] = 1

    EPOCHS = 5
    BATCH_SIZE = 128

    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(70, activation=tf.nn.swish,input_shape=(X_train.shape[1],)))
    model.add(tf.keras.layers.Dense(4, activation=tf.nn.sigmoid))

    # Compile and print out summary of model
    model.compile(loss='categorical_crossentropy', 
                  optimizer='adam',
                 metrics=['accuracy'])

    model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE)
    (test_loss, test_acc) = model.evaluate(X_test, y_test)
    print('Accuracy:', test_acc)
    accs.append(test_acc)
    print("============================================")
    fold_no+=1

In [ ]:
grouped_sev = []
for i in data["Severity"].values:
    if i == 1 or i == 2:
        grouped_sev.append(0)
    else:
        grouped_sev.append(1)

data["Grouped_Severity"] = grouped_sev

In [ ]:
from sklearn.model_selection import StratifiedKFold

labels = data['Grouped_Severity']
X = data.drop(['Severity',"Grouped_Severity"], axis=1)

scaler=MinMaxScaler(feature_range=(0,1))
X = scaler.fit_transform(X)

skf = StratifiedKFold(n_splits=3, random_state=30034, shuffle=False)

fold_no = 1
accs = []  # store each accuracy


for train_index, test_index in skf.split(X, labels):

    # One hot encode labels
    print("FOLD NUMBER = ", str(fold_no))


    X_train = X[train_index]
    X_test = X[test_index]

    y_train = labels[train_index]
    y_test = labels[test_index]

    EPOCHS = 10
    BATCH_SIZE = 128

    
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(40, input_dim=X_train.shape[1], activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE)
    (test_loss, test_acc) = model.evaluate(X_test, y_test)
    print('Accuracy:', test_acc)
    accs.append(test_acc)
    print("============================================")
    fold_no+=1

In [ ]:
data = feather.read_dataframe("{}_numeric_v2.feather".format(FILENAME))

grouped_sev = []
for i in data["Severity"].values:
    if i == 1 or i == 2:
        grouped_sev.append(0)
    else:
        grouped_sev.append(1)

data["Grouped_Severity"] = grouped_sev

from sklearn.model_selection import StratifiedKFold

labels = data['Grouped_Severity']
X = data.drop(['Severity',"Grouped_Severity"], axis=1)

scaler=MinMaxScaler(feature_range=(0,1))
X = scaler.fit_transform(X)

skf = StratifiedKFold(n_splits=3, random_state=30034, shuffle=False)

fold_no = 1
accs = []  # store each accuracy


for train_index, test_index in skf.split(X, labels):

    # One hot encode labels
    print("FOLD NUMBER = ", str(fold_no))


    X_train = X[train_index]
    X_test = X[test_index]

    y_train = labels[train_index]
    y_test = labels[test_index]

    EPOCHS = 10
    BATCH_SIZE = 128

    
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(40, input_dim=X_train.shape[1], activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE)
    (test_loss, test_acc) = model.evaluate(X_test, y_test)
    print('Accuracy:', test_acc)
    accs.append(test_acc)
    print("============================================")
    fold_no+=1

In [ ]:
cols = ['Temperature(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 
        'Wind_Speed(mph)', 'Precipitation(in)',
       'Population_County', 'Drive_County', 'Transit_County', 'Walk_County', 'MedianHouseholdIncome_County']

sns.set_style('darkgrid')
f, axes = plt.subplots(6,2, figsize=(15,20))
y = 0;
for name in cols[1:]:
    i, j = divmod(y, 2)
    sns.boxplot(x=data[name], ax=axes[i, j])
    y = y+1

plt.tight_layout()
plt.show()

### Performing one class SVM for outlier removal

In [ ]:
def outlier_removal(data,columns):
    for col in columns:
        Q1 = np.quantile(data[col].values,0.25)
        Q3 = np.quantile(data[col].values,0.75)
        IQR = Q3-Q1
        
        lower = Q1 - 1.5*IQR
        upper = Q3+ 1.5*IQR
        
        data = data[(data[col]>=lower) & (data[col]<=upper)]
    return data


data = feather.read_dataframe("{}_numeric_v2.feather".format(FILENAME))

numeric_cols = ['Temperature(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 
        'Wind_Speed(mph)', 'Precipitation(in)',
       'Population_County', 'Drive_County', 'Transit_County', 'Walk_County', 'MedianHouseholdIncome_County']
data = outlier_removal(data,numeric_cols).reset_index(drop = True)

In [ ]:
cols = ['Temperature(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 
        'Wind_Speed(mph)', 'Precipitation(in)',
       'Population_County', 'Drive_County', 'Transit_County', 'Walk_County', 'MedianHouseholdIncome_County']

sns.set_style('darkgrid')
f, axes = plt.subplots(6,2, figsize=(15,20))
y = 0;
for name in cols[1:]:
    i, j = divmod(y, 2)
    sns.boxplot(x=new_data[name], ax=axes[i, j])
    y = y+1

plt.tight_layout()
plt.show()

In [ ]:
data['Grouped_Severity'] = np.where(data['Severity']<=2, 0, 1)

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix

labels = data['Grouped_Severity']
X = data.drop(['Severity',"Grouped_Severity",'Precipitation(in)','Visibility(mi)'], axis=1)

scaler=MinMaxScaler(feature_range=(0,1))
X = scaler.fit_transform(X)

skf = StratifiedKFold(n_splits=3, random_state=30034, shuffle=False)

fold_no = 1
accs = []  # store each accuracy


for train_index, test_index in skf.split(X, labels):

    # One hot encode labels
    print("FOLD NUMBER = ", str(fold_no))


    X_train = X[train_index]
    X_test = X[test_index]

    y_train = labels[train_index]
    y_test = labels[test_index]

    EPOCHS = 20
    BATCH_SIZE = 128

    
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(50, input_dim=X_train.shape[1], activation='relu'))
    model.add(tf.keras.layers.Dense(50, input_dim=X_train.shape[1], activation=tf.nn.swish))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE)
    (test_loss, test_acc) = model.evaluate(X_test, y_test)
    print('Accuracy:', test_acc)
    accs.append(test_acc)
    


    y_preds = model.predict(X_test)
    y_preds = np.where(y_preds < 0.5,0,1)


    print("accuracy:", accuracy_score(y_test,y_preds))

    cm=confusion_matrix(y_test,y_preds)
    cm = pd.DataFrame(cm, index = [i for i in "01"],
                      columns = [i for i in "01"])
    cm.index.name = 'Actual'
    cm.columns.name = 'Predicted'
    sns.heatmap(cm, annot=True, 
                cmap='Blues', cbar=False)
    plt.show()
    
    print("============================================")
    fold_no+=1